In [2]:
### ALL NECESSARY LIBRARIES ###
import pandas as pd
import numpy as np
import pickle


In [35]:
## IMPORT DATA ## PREPROCESSED NGRAM FEATURES

with open('preprocessed/persona3_gpt4_responses_030424.pkl', 'rb') as f:
    all_prompts = pickle.load(f)

with open('preprocessed/persona3_gpt4_features_ngrams_030424.pkl', 'rb') as f:
    all_features_df = pickle.load(f)

all_prompts.head(3)

,Instrument,ItemNumber,Dimension,prompt,exclude,no_punct_prompt,generated_text,cleaned_text,preprocessed_text,preprocessed_tokens
0,MEQ30,1,transcendence of time and space,I lost my usual sense of time.,NaN,I lost my usual sense of time,"[Disoriented, Unease in scheduling activities,...","[[unease in scheduling activities, confused ab...","[[unease schedule activity, confuse date day, ...","[[[activity, unease, schedule], [date, day, co..."
1,MEQ30,2,positive mood,I had an experience of amazement.,NaN,I had an experience of amazement,"[Sense of wonder, heightened awareness, emotio...","[[emotional excitement, heightened awareness, ...","[[emotional excitement, heighten awareness, co...","[[[excitement, emotional], [heighten awareness..."
2,MEQ30,3,ineffability,I had a sense that my experience cannot be des...,NaN,I had a sense that my experience cannot be des...,"[Ineffable, overwhelming, complex, beyond lang...","[[mystifying, complex, overwhelming, beyond la...","[[mystify, complex, overwhelm, beyond language...","[[[mystify], [complex], [overwhelm], [language..."


In [36]:
# all_prompts['final_text_noreplacement'] = all_prompts['preprocessed_tokens'].map(lambda row: [' '.join(answer) for answer in row])
all_prompts['final_text_noreplacement'] = all_prompts['preprocessed_tokens'].apply(
    lambda row: [' '.join([' '.join([str(feature) for feature in answer]) for answer in sublist]) for sublist in row])

# all_prompts['final_text_noreplacement'][4]

In [37]:
final_text_noreplacement = all_prompts['final_text_noreplacement']
final_text_noreplacement_combined = final_text_noreplacement.apply(lambda x : ' '.join(x))

final_vocab_noreplacement = list(set(all_prompts['preprocessed_tokens'].explode().explode().explode()))
len(final_vocab_noreplacement)
final_vocab_noreplacement = list(set(all_features_df['feature'].values))

# len(final_vocab_noreplacement)
# len(list(set(all_features_df['feature'].values)))

In [38]:
chosen_features = final_vocab_noreplacement        # CHOOSE FULL REPLACEMENT/NO REPLACEMENT/THRESHOLDED HERE
chosen_features = [item for item in chosen_features if not isinstance(item, float)]     # REMOVE FLOAT/NUMBERS/NANS
final_text = final_text_noreplacement_combined     # CHOOSE FULL REPLACEMENT/NO REPLACEMENT/THRESHOLDED HERE
len(chosen_features)

4471

In [39]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

count_vect = CountVectorizer(vocabulary=chosen_features, ngram_range=(1, 3), token_pattern=r"(?u)\b\w[\w-]*\w\b")
tfidf_vect = TfidfVectorizer(vocabulary=chosen_features, ngram_range=(1, 3), token_pattern=r"(?u)\b\w[\w-]*\w\b")
bow_matrix = count_vect.fit_transform(final_text)
tfidf_matrix = tfidf_vect.fit_transform(final_text)
# np.sum(bow_matrix[0])

bow_matrix.shape

(1023, 4471)

In [40]:
# Get the occurrence frequency of each feature (word)
word_frequencies = np.transpose(bow_matrix.sum(axis=0))

frequency_df = pd.DataFrame(columns=['feature', 'frequency'])
frequency_df['feature'] = count_vect.get_feature_names_out()
frequency_df['frequency'] = word_frequencies
frequency_df.head()

,feature,frequency
0,rushy,1
1,multiplicity,4
2,disproportionate,1
3,severe,2
4,trait,5


In [41]:
# VISUALIZE FEATURE FREQUENCIES
import plotly.graph_objs as go
import pandas as pd

# Sort the words and counts based on counts in descending order
df_sorted = frequency_df.sort_values(by='frequency', ascending=False)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sorted['feature'], y=df_sorted['frequency'], mode='lines', name='Feature Occurrence Frequency'))

fig.update_layout(
    title='Feature Occurrence Frequency',
    xaxis_title='Features',
    yaxis_title='Frequency',
)

In [42]:
df_sorted.head(10)

#df_sorted[df_sorted['feature'] == 'ization']

,feature,frequency
2896,emotional,2097
2557,perception,1891
203,sense,1618
2376,heighten,1367
1294,sensation,1220
3215,awareness,1116
957,thought,1057
3768,mental,1033
994,focus,1016
79,physical,940


In [43]:
# remove frequent features FIRST?
# Remove the top 6 rows
df_removed = df_sorted.iloc[6:]
df_removed.head(10)

,feature,frequency
957,thought,1057
3768,mental,1033
994,focus,1016
79,physical,940
1482,sensory,688
3338,heart,643
4346,increase,622
3848,cognitive,587
2340,lack,529
2957,time,511


In [44]:
##### USE THIS: REMOVE BY ONLY KEEPING A CERTAIN PERCENTAGE
#df_sorted = df_removed

word_counts = df_sorted.set_index('feature')['frequency'].to_dict()

total_word_count = df_sorted['frequency'].sum()
total_word_count

# Step 3: Iterate through the sorted DataFrame to find the words that make up 75% (choose percentile)
threshold = 0.8 * total_word_count
cumulative_sum = 0
selected_words = {}
filtered_words = {}

for index, row in df_sorted.iterrows():
    cumulative_sum += row['frequency']
    # selected_words.append(row['feature'])
    # if cumulative_sum >= threshold:
    #     break
    if cumulative_sum <= threshold:
        selected_words[row['feature']] = row['frequency']
    else:
        filtered_words[row['feature']] = row['frequency']

# Step 4: Extract the selected words
selected_df = df_sorted[df_sorted['feature'].isin(list(selected_words.keys()))]

# Print the selected words and their frequencies
len(selected_df)

548

In [45]:
# REMOVE RESPONSES WITH INFREQUENT FEATURES (AFTER SYNONYMIZING)
# all_prompts['features_extracted_thresholded_filtered'] = all_prompts['features_extracted_thresholded'].apply(
all_prompts['features_filtered_tokens'] = all_prompts['preprocessed_tokens'].apply(
    lambda sublists: [[sublist for sublist in inner_list if not any(element in sublist for element in list(filtered_words.keys()))] for inner_list in sublists])

In [46]:
### Number of responses removed
n_answers_before = all_prompts['preprocessed_tokens'].apply(lambda x: sum(len(sublist) for sublist in x)).sum()
n_answers_after = all_prompts['features_filtered_tokens'].apply(lambda x: sum(len(sublist) for sublist in x)).sum()

print("n responses before filtering: ", n_answers_before)
print("n responses after filtering: ", n_answers_after)
print("n responses removed: ", n_answers_before-n_answers_after)

n responses before filtering:  25600
n responses after filtering:  13536
n responses removed:  12064


In [47]:
# and then reconstruct the documents
# all_prompts['final_text_thresholded_filtered'] = all_prompts['features_extracted_thresholded_filtered'].apply(
all_prompts['final_text_filtered'] = all_prompts['features_filtered_tokens'].apply(
    lambda row: [' '.join([' '.join([str(feature) for feature in answer]) for answer in sublist]) for sublist in row])

# final_text_thresholded_filtered = all_prompts['final_text_thresholded_filtered']                  # this should be a list with len(list) = n if n >1
final_text_thresholded_filtered = all_prompts['final_text_filtered']                  # this should be a list with len(list) = n if n >1
final_text_thresholded_filtered_combined = final_text_thresholded_filtered.apply(lambda x : ' '.join(x))

final_text = final_text_thresholded_filtered_combined

In [48]:
# REMAKE BOW MATRIX
# final_features = filtered_final_vocab
final_features = list(set(selected_df['feature'].values))
## this part doesn't necessarily need to be here, it's also in next notebook
count_vect_filtered = CountVectorizer(vocabulary=final_features, ngram_range=(1, 3), token_pattern=r"(?u)\b\w[\w-]*\w\b")
bow_matrix_filtered = count_vect_filtered.fit_transform(final_text)

bow_matrix_filtered.shape

(1023, 548)

In [49]:
feature_names = count_vect_filtered.get_feature_names_out()
len(feature_names)

548

In [50]:
# save everything
with open('feature filtering/persona3_feat_app_matrix_gpt4_030424.pkl', 'wb') as f:
    pickle.dump(all_prompts, f)

with open('feature filtering/persona3_final_features_filtered_gpt4_030424.pkl', 'wb') as f:
    pickle.dump(final_features, f)

with open('feature filtering/persona3_final_documents_gpt4_030424.pkl', 'wb') as f:
    pickle.dump(final_text, f)